In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import process, fuzz

import multiprocessing as mp
from functools import partial
from itertools import repeat

from run_match import get_region_match


In [2]:
# load the data
def load_data():
    ex = pd.read_csv("ex.csv",delimiter=";")
    ex.columns = ["ext_original_name", "ext_nation", "ext_region"]

    inter = pd.read_csv("int.csv",delimiter=";")
    inter.columns = ["customer_id","int_original_name", "int_nation", "int_region"]

    # remove the company type from the name
    ex['clean_name'] = remove_words(ex['ext_original_name'])

    # inter split the region from name
    int_name_region = inter["int_original_name"].str.split(',', expand=True)
    inter['clean_name'] = remove_words(int_name_region[0])
    # inter["int_sub_region"] = int_name_region[1]
    del int_name_region

    # drop the duplicated rows
    clean_ex = ex.drop_duplicates()
    clean_int = inter.drop_duplicates()

    # reomve the unkonow company
    clean_ex = clean_ex.loc[clean_ex["clean_name"]!="unknown", :]
    clean_int = clean_int.loc[clean_int["clean_name"]!="unknown", :]
    return clean_ex, clean_int


def remove_words(origina_name_series): 
    # lower the word
    origina_name_series = origina_name_series.str.lower()
    
    # remove the special characers
    origina_name_series = origina_name_series.str.replace('[^\w\s]', '', regex=True)
        
    remove_words = ["llc", "ltd", 'limited',"co", 'corp',"inc",'bv',
                    'holding','holdings', 'plc', 'group', 'bvba','sa']
    remove_words = r'\b(?:{})\b'.format('|'.join(remove_words))
    new_name_series = origina_name_series.str.replace(remove_words, '', regex=True)
    
    # remove the extra space
    new_name_series = new_name_series.str.replace(r'\s+',' ', regex=True)
    new_name_series = new_name_series.str.strip()
    
    return new_name_series


# count the freq words
def get_word_freq(series, top=20):
    return pd.Series(' '.join(series).split()).value_counts()[:top]


In [4]:
def run_match(clean_ex, clean_int):
    match_ration_dic = {"ratio":fuzz.ratio,
                        "token_sort_ratio":fuzz.token_sort_ratio}

    all_region_result = []
    for region, single_region_ex in tqdm(clean_ex.groupby("ext_region")):
        region_int = clean_int.loc[clean_int["int_region"]==region]
        
        # mutil-core match
        n_cores = (mp.cpu_count() - 1)
        split_df = np.array_split(single_region_ex, n_cores)
        in_arg = []
        for i in range(len(split_df)):
            if split_df[i].size >0:
                in_arg.append([split_df[i], region_int, match_ration_dic])
            else:
                in_arg.append([pd.DataFrame(columns=single_region_ex.columns),
                             region_int, match_ration_dic])

        with mp.Pool(processes = n_cores) as multi_process:
            results = multi_process.starmap(get_region_match, in_arg)
        single_region_rs = pd.concat(results)
        assert len(single_region_rs) == len(single_region_ex)
        
        single_region_rs = single_region_rs.merge(
                  region_int, 
                  left_on = f"final_match", 
                  right_on = "clean_name",
                  how="left",
                  suffixes=('', f'_matchedby')).drop(
                        f'clean_name_matchedby',axis=1)
        all_region_result.append(single_region_rs)

    # concat different region results
    match_df = pd.concat(all_region_result)
    
    # save the result
    print("save results ...")
    internal_info_col = ["final_match","customer_id", "int_original_name", "int_nation", "int_region"]
    final_mathch = match_df[clean_ex.columns.tolist() + internal_info_col].dropna(
        subset=["final_match"], how='all').drop(["clean_name","final_match"],axis=1)

    
    return final_mathch, match_df

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]

(4, 9)

In [ ]:
clean_ex, clean_int = load_data()

final_mathched, match_details = run_match(clean_ex, clean_int)

final_mathched.shape

In [10]:
final_mathched.to_excel("final_mathched.xlsx",index=False) 
match_result_info.to_excel("match_result_info.xlsx",index=False) 

In [11]:
final_mathched.drop(["clean_name","final_match"],axis=1).to_excel("final_mathched.xlsx",index=False) 